In [64]:
import pandas as pd
import numpy as np
from scipy.spatial import distance, distance_matrix
from scipy.spatial.distance import pdist
from pandas import ExcelWriter
from pandas import ExcelFile

In [65]:
X_8 = pd.read_excel('8clusters.xlsx', sheetname='Data')
X_12 = pd.read_excel('12clusters.xlsx', sheetname='Data')
X_14 = pd.read_excel('14clusters.xlsx', sheetname='Data')
datasets = [X_8, X_12, X_14]

In [66]:
def centrs_cluster(X):
    data = X.iloc[:,2:]
    return [data.loc[X.iloc[:,1] == i].mean().as_matrix() 
            for i in sorted(X.iloc[:,1].unique())]

def differences(X, m, metric):
    return [metric(u,m) for _, u in X.iterrows()]

def nCr(n,r):
    import math
    f = math.factorial
    return f(n) / f(r) / f(n-r)
    
def up(X, metric):
    clusters = X_8.iloc[:,1].unique()
    centres = centrs_cluster(X)
    data = X.iloc[:,2:]
    return np.mean([np.mean( differences(data.loc[X.iloc[:,1] == c], centres[c-1], metric) )
                    for c in clusters])

def down(X, metric="euclidean"):
    clusters = X_8.iloc[:,1].unique()
    centres = centrs_cluster(X)
    return sum(pdist(centrs_cluster(X), metric=metric))/nCr(len(clusters), 2)

In [67]:
inside_group = np.array([up(X, distance.euclidean) for X in datasets])
outside_group = np.array([down(X) for X in datasets])
devided = inside_group/outside_group

pd.DataFrame([["inside", *inside_group, min(inside_group)], 
              ["outside", *outside_group, max(outside_group)], 
              ["devided", *devided, min(devided)]], 
              columns=["name", "8 cluster", "12 cluster", "14 cluster", "better"])

,name,8 cluster,12 cluster,14 cluster,better
0,inside,36.842863,35.424134,35.454535,35.424134
1,outside,66.920037,180.573399,251.213317,251.213317
2,devided,0.550551,0.196176,0.141133,0.141133


In [68]:
inside_group = np.array([up(X, distance.sqeuclidean) for X in datasets])
outside_group = np.array([down(X, "sqeuclidean") for X in datasets])
devided = inside_group/outside_group

pd.DataFrame([["inside", *inside_group, min(inside_group)], 
              ["outside", *outside_group, max(outside_group)], 
              ["devided", *devided, min(devided)]], 
              columns=["name", "8 cluster", "12 cluster", "14 cluster", "better"])

,name,8 cluster,12 cluster,14 cluster,better
0,inside,1542.635191,1408.393874,1414.321541,1408.393874
1,outside,4964.938492,16404.086635,22832.794843,22832.794843
2,devided,0.310706,0.085856,0.061943,0.061943


# Normalized data

In [69]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

def norm_data(X):
    x_scaled = min_max_scaler.fit_transform(X.iloc[:,2:].values)
    X.iloc[:,2:] = x_scaled
    return X

In [70]:
scaled_datasets = list(map(norm_data, datasets))

inside_group = np.array([up(X, distance.euclidean) for X in scaled_datasets])
outside_group = np.array([down(X) for X in scaled_datasets])
devided = inside_group/outside_group

pd.DataFrame([["inside", *inside_group, min(inside_group)], 
              ["outside", *outside_group, max(outside_group)], 
              ["devided", *devided, min(devided)]], 
              columns=["name", "8 cluster", "12 cluster", "14 cluster", "better"])

/nix/store/i9yk45439h7wk5syf1s3d4arx56v6wqk-python3.6-scikit-learn-0.19.0/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,name,8 cluster,12 cluster,14 cluster,better
0,inside,0.400438,0.386218,0.385696,0.385696
1,outside,0.721546,1.969706,2.723437,2.723437
2,devided,0.554972,0.196079,0.141621,0.141621


In [71]:
inside_group = np.array([up(X, distance.sqeuclidean) for X in datasets])
outside_group = np.array([down(X, "sqeuclidean") for X in datasets])
devided = inside_group/outside_group

pd.DataFrame([["inside", *inside_group, min(inside_group)], 
              ["outside", *outside_group, max(outside_group)], 
              ["devided", *devided, min(devided)]], 
              columns=["name", "8 cluster", "12 cluster", "14 cluster", "better"])

,name,8 cluster,12 cluster,14 cluster,better
0,inside,0.181106,0.166363,0.166106,0.166106
1,outside,0.579323,1.961136,2.674454,2.674454
2,devided,0.312617,0.084830,0.062108,0.062108
